In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
# Loading the dataset
data = pd.read_json("C:/Users/PRABHA/OneDrive/Documents/recipe-recommender/train.json")
data.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [6]:
print("Dataset Info:")
print(data.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           39774 non-null  int64 
 1   cuisine      39774 non-null  object
 2   ingredients  39774 non-null  object
dtypes: int64(1), object(2)
memory usage: 932.3+ KB
None


In [7]:
print(data.columns)

Index(['id', 'cuisine', 'ingredients'], dtype='object')


In [8]:
#checking for missing values 
missing_values = data.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
id             0
cuisine        0
ingredients    0
dtype: int64


In [9]:
#standardizing the ingredients for smoother functioning 
data["ingredients"] = data["ingredients"].apply(lambda x: [i.lower().strip() for i in x])
print(data["ingredients"].head())


0    [romaine lettuce, black olives, grape tomatoes...
1    [plain flour, ground pepper, salt, tomatoes, g...
2    [eggs, pepper, salt, mayonaise, cooking oil, g...
3                  [water, vegetable oil, wheat, salt]
4    [black pepper, shallots, cornflour, cayenne pe...
Name: ingredients, dtype: object


In [10]:
import tkinter as tk
from tkinter import messagebox
import requests

In [11]:
# Constants
API_KEY = '30513744673749ee8167cef8d3faab23'
BASE_URL = 'https://api.spoonacular.com/recipes/'

In [12]:
# Function to fetch recipes by ingredients
def fetch_recipes():
    user_ingredients = entry_ingredients.get()
    if not user_ingredients.strip():
        messagebox.showerror("Input Error", "Please enter some ingredients.")
        return

    # API call to fetch recipes
    url = f"{BASE_URL}findByIngredients?apiKey={API_KEY}"
    params = {
        "ingredients": user_ingredients,
        "number": 5,  # Fetch top 5 recipes
        "ranking": 2
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        recipes = response.json()
        display_recipes(recipes)
    else:
        messagebox.showerror("API Error", f"Failed to fetch recipes. Error: {response.status_code}")

In [13]:
# Function to display recipe recommendations
def display_recipes(recipes):
    recipe_window = tk.Toplevel(root)
    recipe_window.title("Recipe Recommendations")

    tk.Label(recipe_window, text="Here are your recommended recipes:", font=("Helvetica", 12, "bold")).pack(pady=10)

    for idx, recipe in enumerate(recipes):
        recipe_text = f"{idx + 1}. {recipe['title']} (Uses {recipe['usedIngredientCount']} ingredients, {recipe['missedIngredientCount']} missing)"
        tk.Label(recipe_window, text=recipe_text, wraplength=400, justify="left").pack(anchor="w", padx=10)

    tk.Label(recipe_window, text="\nEnter the recipe number for details:", font=("Helvetica", 10)).pack()
    recipe_choice = tk.Entry(recipe_window)
    recipe_choice.pack()

    def get_recipe_details():
        choice = recipe_choice.get()
        if not choice.isdigit() or int(choice) < 1 or int(choice) > len(recipes):
            messagebox.showerror("Input Error", "Please select a valid recipe number.")
            return
        selected_recipe_id = recipes[int(choice) - 1]['id']
        fetch_recipe_details(selected_recipe_id)

    tk.Button(recipe_window, text="Get Recipe Details", command=get_recipe_details).pack(pady=10)

In [14]:
# Function to fetch full recipe details
def fetch_recipe_details(recipe_id):
    url = f"{BASE_URL}{recipe_id}/information?apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        display_recipe_details(data)
    else:
        messagebox.showerror("API Error", f"Failed to fetch recipe details. Error: {response.status_code}")

In [15]:
# Function to display full recipe details
def display_recipe_details(data):
    details_window = tk.Toplevel(root)
    details_window.title(data['title'])

    # Display recipe title
    tk.Label(details_window, text=data['title'], font=("Helvetica", 16, "bold")).pack(pady=10)

    # Display prep time and servings
    tk.Label(details_window, text=f"Prep Time: {data.get('readyInMinutes', 'N/A')} minutes").pack()
    tk.Label(details_window, text=f"Servings: {data.get('servings', 'N/A')}").pack()

    # Display ingredients
    tk.Label(details_window, text="\nIngredients:", font=("Helvetica", 12, "bold")).pack(anchor="w", padx=10)
    for ingredient in data.get('extendedIngredients', []):
        tk.Label(details_window, text=f"- {ingredient['name'].capitalize()}", wraplength=400, justify="left").pack(anchor="w", padx=20)

    # Display instructions
    tk.Label(details_window, text="\nInstructions:", font=("Helvetica", 12, "bold")).pack(anchor="w", padx=10)
    instructions = data.get('instructions', "No instructions available.")
    tk.Label(details_window, text=instructions, wraplength=400, justify="left").pack(anchor="w", padx=20)



In [16]:
# Main GUI
root = tk.Tk()
root.title("Recipe Recommendation System")
root.geometry("500x400")

''

In [17]:
# UI Elements
tk.Label(root, text="Welcome to the Recipe Recommendation System!", font=("Helvetica", 14, "bold")).pack(pady=20)
tk.Label(root, text="Enter your available ingredients (comma-separated):", font=("Helvetica", 10)).pack(pady=10)
entry_ingredients = tk.Entry(root, width=50)
entry_ingredients.pack()

tk.Button(root, text="Find Recipes", command=fetch_recipes).pack(pady=20)

In [18]:
# Start the main loop
root.mainloop()